In [2]:
!pip install requests beautifulsoup4 psycopg2-binary python-dotenv


In [1]:
import os
import json
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup
import psycopg2
from psycopg2.extras import execute_values
from dotenv import load_dotenv

load_dotenv()

LOCAL_BRONZE_PATH = "bronze_jobs.json"
LOCAL_SCRAPED_SAVE_PATH = "scraped_raw.json"
MAX_JOB_AGE_DAYS = 7
KEYWORDS = ["engineer", "software", "developer", "audit", "analyst"]


In [2]:
def save_json(path, data):
    with open(path, "w") as f:
        json.dump(data, f, indent=2)

def load_json(path):
    if not os.path.exists(path):
        return []
    with open(path, "r") as f:
        return json.load(f)

from datetime import datetime, timezone

def timestamp():
    return datetime.now(timezone.utc).isoformat()



In [25]:
import re
from bs4 import BeautifulSoup
import requests

BASE = "https://www.governmentjobs.com"
JOB_URL_RE = re.compile(r"^/jobs/\d")  # href starts with /jobs/<digit>

def search_govjobs(keyword: str):
    url = f"{BASE}/jobs?keyword={keyword}"
    print("Searching:", url)

    resp = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    jobs = []

    # Look for ANY anchor that looks like a job detail URL
    for a in soup.find_all("a", href=True):
        href = a["href"]
        if not JOB_URL_RE.match(href):
            continue

        title = a.get_text(strip=True)
        # Quick sanity check to skip garbage links
        if not title or "Job Alert" in title or "Loading Job Details" in title:
            continue

        full_url = BASE + href
        jobs.append({
            "job_url": full_url,
            "title": title,
        })

    print(f"  found {len(jobs)} job URLs for keyword='{keyword}'")
    return jobs




In [26]:
search_govjobs("software")[:5]


Searching: https://www.governmentjobs.com/jobs?keyword=software
  found 10 job URLs for keyword='software'


[{'job_url': 'https://www.governmentjobs.com/jobs/5148422-0/software-application-specialist',
  'title': 'Software Application Specialist'},
 {'job_url': 'https://www.governmentjobs.com/jobs/120764-1/software-development-part-time-faculty',
  'title': 'Software Development, Part time Faculty'},
 {'job_url': 'https://www.governmentjobs.com/jobs/5126271-0/software-analyst-i-ii-onsite-only-business-and-services-track',
  'title': 'Software Analyst I/II (Onsite Only) (Business and Services Track)'},
 {'job_url': 'https://www.governmentjobs.com/jobs/5144458-0/senior-software-developer',
  'title': 'Senior Software Developer*'},
 {'job_url': 'https://www.governmentjobs.com/jobs/5041458-0/senior-software-developer',
  'title': 'Senior Software Developer'}]

In [27]:
import re

CONTROL_CHARS = re.compile(r"[\x00-\x08\x0b\x0c\x0e-\x1f\x7f]")

def extract_salary_text(soup):
    """
    Try to pull the salary/base pay text from the job page HTML.
    Returns a raw string like '$95,000 - $120,000 Annually' or None.
    """
    # Try JSON-LD first (if it has baseSalary)
    script = soup.find("script", {"type": "application/ld+json"})
    if script and script.string:
        try:
            data = json.loads(script.string, strict=False)
            if isinstance(data, list):
                data = data[0]
            base_salary = data.get("baseSalary")
            if isinstance(base_salary, dict):
                # e.g. {"@type": "MonetaryAmount","value": {"minValue":...,"maxValue":...,"unitText":"YEAR"}}
                val = base_salary.get("value", {})
                min_val = val.get("minValue")
                max_val = val.get("maxValue")
                unit = val.get("unitText")
                if min_val and max_val:
                    return f"{min_val} - {max_val} {unit}"
                elif min_val:
                    return f"{min_val} {unit}"
        except Exception:
            pass  # fall through to HTML heuristics

    # Heuristic HTML parsing: look for labels containing "Salary"
    label = soup.find(string=re.compile(r"salary", re.I))
    if label:
        # Walk up to a container row
        row = label.find_parent(["tr", "p", "div", "dt", "li"])
        if row:
            # Look for the 'value' near the label
            # Table row: <tr><th>Salary</th><td>$95,000 - $120,000 Annually</td></tr>
            cell = row.find("td")
            if cell:
                return cell.get_text(strip=True)
            # Definition list: <dt>Salary</dt><dd>...</dd>
            sib = row.find_next_sibling()
            if sib:
                return sib.get_text(strip=True)

    return None


def scrape_job_detail(url):
    resp = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    script = soup.find("script", {"type": "application/ld+json"})
    if not script or not script.string:
        return None

    raw = CONTROL_CHARS.sub(" ", script.string)
    data = json.loads(raw, strict=False)
    if isinstance(data, list):
        data = data[0]

    salary_text = extract_salary_text(soup)

    return {
        "job_id": url.rstrip("/").split("/")[-2],
        "title": data.get("title"),
        "company": data.get("hiringOrganization", {}).get("name", ""),
        "location": data.get("jobLocation", {}).get("address", {}).get("addressLocality", ""),
        "description": data.get("description", ""),
        "posted_at": data.get("datePosted", timestamp()),
        "url": url,
        "job_base_pay_range": salary_text,   # 🔴 now this exists in raw
    }





In [28]:
def scrape_all_keywords():
    results = []
    seen = set()

    for kw in KEYWORDS:
        search_results = search_govjobs(kw)
        for job in search_results:
            url = job["job_url"]
            if url in seen:
                continue

            seen.add(url)

            try:
                detail = scrape_job_detail(url)
                if detail:
                    detail["scraped_at"] = timestamp()
                    results.append(detail)
            except Exception as e:
                print("Error scraping detail:", url, e)

    print(f"Total jobs scraped: {len(results)}")
    return results

scraped_jobs = scrape_all_keywords()
save_json("scraped_raw.json", scraped_jobs)


Searching: https://www.governmentjobs.com/jobs?keyword=engineer
  found 10 job URLs for keyword='engineer'
Searching: https://www.governmentjobs.com/jobs?keyword=software
  found 10 job URLs for keyword='software'
Searching: https://www.governmentjobs.com/jobs?keyword=developer
  found 10 job URLs for keyword='developer'
Searching: https://www.governmentjobs.com/jobs?keyword=audit
  found 10 job URLs for keyword='audit'
Searching: https://www.governmentjobs.com/jobs?keyword=analyst
  found 10 job URLs for keyword='analyst'
Total jobs scraped: 48


In [29]:
# Global config (re-run this after any kernel restart)

BRONZE_PATH = "bronze_jobs.json"
SILVER_PATH = "silver_jobs.json"
GOLD_PATH = "gold_jobs.json"

# ONLY needed if you're using keyword scraping
KEYWORDS = ["engineer", "software", "developer", "audit", "analyst"]


In [30]:
# Load bronze
bronze = load_json(BRONZE_PATH)

# Append today's scrape
bronze.extend(scraped_jobs)
save_json(BRONZE_PATH, bronze)

# ---- Silver: latest job version ----
latest = {}
for job in bronze:
    key = job["job_id"]
    ts = datetime.fromisoformat(job["scraped_at"])
    if key not in latest or ts > datetime.fromisoformat(latest[key]["scraped_at"]):
        latest[key] = job

silver = list(latest.values())

# ---- Gold: only recent postings ----
from datetime import datetime, timedelta, timezone

# ---- Gold: only recent postings ----
cutoff = datetime.now(timezone.utc) - timedelta(days=MAX_JOB_AGE_DAYS)

def parse_ts(ts_str: str) -> datetime:
    dt = datetime.fromisoformat(ts_str)
    # If for some reason it's naive, assume UTC
    if dt.tzinfo is None:
        dt = dt.replace(tzinfo=timezone.utc)
    return dt

gold = [
    j for j in silver
    if parse_ts(j["scraped_at"]) >= cutoff
]


print("Bronze:", len(bronze))
print("Silver:", len(silver))
print("Gold (active):", len(gold))

save_json("silver_jobs.json", silver)
save_json("gold_jobs.json", gold)



Bronze: 192
Silver: 55
Gold (active): 55


In [31]:
import pandas as pd

df_gold = pd.DataFrame(gold)
df_gold



,job_id,title,company,location,description,posted_at,url,job_base_pay_range,scraped_at
0,5108053-0,County Engineer,New Hanover County,"New Hanover County, NC","New Hanover County is seeking an experienced, ...",2025-10-13,https://www.governmentjobs.com/jobs/5108053-0/...,111570 - 186699 YEAR,2025-11-25T19:31:08.510627+00:00
1,5121231-0,Engineer (Planning),Orange County Sanitation District,"CA 92708, CA",&lt;p style=&quot;margin: 0&quot;&gt;&lt;span ...,2025-10-29,https://www.governmentjobs.com/jobs/5121231-0/...,142521.6 - 173243.2 YEAR,2025-11-25T19:31:08.955357+00:00
2,4912084-0,Senior Engineer,Klickitat Public Utility,"Goldendale, WA 98620","This position is responsible to plan, scope, s...",2025-04-17,https://www.governmentjobs.com/jobs/4912084-0/...,135826 - 183764 YEAR,2025-11-25T19:31:09.453946+00:00
3,5128516-0,Senior Civil Engineer,City of Davis,"Davis, CA",&lt;p style=&quot;margin: 0 0 0.0001pt; text-a...,2025-10-30,https://www.governmentjobs.com/jobs/5128516-0/...,123896.656 - 150596.784 YEAR,2025-11-25T19:31:09.843842+00:00
4,5117584-0,ENGINEER 1/ENGINEER 2 SEWER UTILITIES,Kitsap County,"Bremerton, WA",&lt;div style=&quot;text-align: left&quot;&gt;...,2025-11-04,https://www.governmentjobs.com/jobs/5117584-0/...,84260.8 - 123406.4 YEAR,2025-11-25T19:31:10.211951+00:00
5,5109149-0,Electrical Engineer III - Systems,Lewis County Public Utility District,"Chehalis, WA",&lt;p&gt;The Electrical Engineer III - Systems...,2025-10-10,https://www.governmentjobs.com/jobs/5109149-0/...,114310 - 171466 YEAR,2025-11-25T19:31:10.581012+00:00
6,4880582-0,Senior Civil Engineer - Solid Waste,City of Winston-Salem,"Winston-Salem, NC",&lt;p&gt;Performs difficult professional work ...,2025-03-23,https://www.governmentjobs.com/jobs/4880582-0/...,87336 - 111328 YEAR,2025-11-25T19:31:10.949540+00:00
7,4937456-0,CIVIL ENGINEER - CIP ($5000 Hiring Incentive) ...,City of Surprise,"Surprise, AZ",&lt;div style=&quot;text-align: left&quot;&gt;...,2025-06-20,https://www.governmentjobs.com/jobs/4937456-0/...,84881.16 - 127321.74 YEAR,2025-11-25T19:31:11.406918+00:00
8,5134955-0,Engineer,Travis County,"Austin, TX",&lt;p&gt;Functions as a project manager over h...,2025-11-09,https://www.governmentjobs.com/jobs/5134955-0/...,86405.46 - 112327.09 YEAR,2025-11-25T19:31:11.782429+00:00
9,129910-1,Engineer,City of Barberton,"Barberton, OH",&lt;p&gt;The City of Barberton is now acceptin...,2025-09-05,https://www.governmentjobs.com/jobs/129910-1/e...,74647.51 - 111063.66 YEAR,2025-11-25T19:31:12.175784+00:00


# Persistence to supabase

In [32]:
import requests
import numpy as np
import pandas as pd
from datetime import datetime

SUPABASE_URL = "https://xaooqthrquigpwpsbmss.supabase.co"
SUPABASE_KEY = ""
TABLE_NAME = "job_listings"
SUPABASE_REST_URL = f"{SUPABASE_URL}/rest/v1/{TABLE_NAME}"

# Exact columns Supabase says exist:
JOB_LISTINGS_COLUMNS = [
    "apply_link",
    "company_name",
    "competitiveness_score",
    "country_code",
    "id",
    "job_base_pay_range",
    "job_employment_type",
    "job_industries",
    "job_location",
    "job_posted_date",
    "job_seniority_level",
    "job_summary",
    "job_title",
    "url",
]


def json_safe(obj):
    if isinstance(obj, np.generic):
        return obj.item()
    if isinstance(obj, (pd.Timestamp, datetime)):
        return obj.isoformat()
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    if isinstance(obj, list):
        return [json_safe(v) for v in obj]
    if isinstance(obj, dict):
        return {k: json_safe(v) for k, v in obj.items()}
    return obj


def map_gold_row_to_job_listing(r: dict) -> dict:
    """
    Map one row from `gold` into the `job_listings` schema.
    Adjust this if your gold keys are different.
    """
    return {
        # Both url + apply_link point to the job page
        "url": r.get("url"),
        "apply_link": r.get("url"),

        "job_title": r.get("job_title") or r.get("title"),
        "company_name": r.get("company_name") or r.get("company"),
        "job_location": r.get("job_location") or r.get("location"),

        # If you have a competitive score in gold, map it; else None
        "competitiveness_score": r.get("competitiveness_score") or r.get("competitive_score"),

        # Defaults / not present in gold yet:
        "country_code": r.get("country_code") or None,
        "job_seniority_level": r.get("job_seniority_level") or None,
        "job_employment_type": r.get("job_employment_type") or None,
        "job_industries": r.get("job_industries") or None,
        "job_base_pay_range": r.get("job_base_pay_range") or None,

        # Summary/description
        "job_summary": r.get("job_summary") or r.get("description"),

        # Posted date (use gold['posted_at'] if present)
        "job_posted_date": r.get("job_posted_date") or r.get("posted_at"),

        # id is serial in Supabase, so we don't send it (let DB generate)
        "id": None,
    }


def upload_gold_to_supabase(gold, chunk_size=500):
    """
    Uploads the 'gold' table to Supabase job_listings via REST API.

    gold: list[dict] or pandas.DataFrame
    """
    # Normalize to list[dict]
    if isinstance(gold, pd.DataFrame):
        df = gold.where(pd.notnull(gold), None)
        raw_records = df.to_dict(orient="records")
    else:
        raw_records = []
        for r in gold:
            clean = {}
            for k, v in r.items():
                if isinstance(v, float) and np.isnan(v):
                    v = None
                clean[k] = v
            raw_records.append(clean)

    if not raw_records:
        print("No records in gold to upload.")
        return

    # Map to job_listings schema
    mapped = [map_gold_row_to_job_listing(r) for r in raw_records]

    # Drop 'id' so Supabase can auto-generate it
    for m in mapped:
        m.pop("id", None)

    # JSON-safe cleanup
    mapped = [json_safe(m) for m in mapped]

    headers = {
        "apikey": SUPABASE_KEY,
        "Authorization": f"Bearer {SUPABASE_KEY}",
        "Content-Type": "application/json",
    }

    total = len(mapped)
    print(f"Uploading {total} rows to {TABLE_NAME} via REST...")

    for i in range(0, total, chunk_size):
        batch = mapped[i : i + chunk_size]
        if not batch:
            continue

        resp = requests.post(
            SUPABASE_REST_URL,
            headers={**headers, "Prefer": "return=minimal"},
            json=batch,
            timeout=15,
        )
        if not resp.ok:
            print(f"Batch {i}-{i+len(batch)} failed:", resp.status_code, resp.text)
        else:
            print(f"Batch {i}-{i+len(batch)} uploaded.")




In [49]:
import html
import re

def map_gold_row_to_job_listing(r: dict) -> dict:
    location = r.get("location") or ""
    # super dumb example: US if it has ", CA" or ", NY" etc.
    country_code = "US" 

    title = (r.get("job_title") or r.get("title") or "").lower()
    desc = (r.get("job_summary") or r.get("description") or "").lower()
    if "intern" in title or "junior" in title or "associate" or "engineer i":
        seniority = "Entry level"
    elif "senior" in title or "engineer iii" in title:
        seniority = "Senior"
    else:
        seniority = "Mid"
    industries = []
    if "engineer" in title or "engineer" in desc:
        industries += ["Engineering"]
    if "software" in title or "software" in desc:
        industries += ["Software"]
    if "data" in title or "data" in desc:
        industries += ["Data Science"]
    elif "analyst" in title or "analyst" in desc:
        industries += ["Business Analysis"]
    if "government" in title or "government" in desc:
        industries += ["Government"]
    if "AI" in title or "AI" in desc or ("machine" in title and "learning" in title) or ("machine" in desc and "learning" in desc):
        industries += ["Artificial Intelligence"]
    job_type = "fed"
    if "intern" in title or "intern" in desc:
        job_type = "internship"
    if "season" in title or "season" in desc:
        job_type = "seasonal"
    if "full-time" in title or "full-time" in desc or "full time" in desc or "full time" in title:
        job_type = "full-time"
    if "contract" in title or "contract" in desc:
        job_type = "contractor"
    return {
        "url": r.get("url"),
        "apply_link": r.get("url"),

        "job_title": r.get("job_title") or r.get("title"),
        "company_name": r.get("company_name") or r.get("company"),
        "job_location": location,

        "competitiveness_score": r.get("competitiveness_score") or r.get("competitive_score"),

        "country_code": country_code,
        "job_seniority_level": seniority,
        "job_employment_type": job_type,   # fill from source if you have it
        "job_industries": industries,
        "job_base_pay_range": r.get("job_base_pay_range") or None,

        "job_summary": r.get("job_summary") or r.get("description"),
        "job_posted_date": r.get("job_posted_date") or r.get("posted_at"),

        "id": None,
    }


def clean_summary(text):
    if text is None:
        return None
    decoded = html.unescape(str(text))
    cleaned = re.sub(r"<[^>]+>", "", decoded)
    return cleaned.strip()
    
def clean_html_text(text):
    if not text:
        return None
    
    # Fix broken entities like "&nb." → " "
    text = re.sub(r"&nb[^;]*;?", " ", text)
    
    # Decode proper HTML entities (&nbsp;, &amp;, etc.)
    decoded = html.unescape(text)

    # Strip extra whitespace
    return decoded.strip()
    
gold_clean = []
for row in gold:  # gold is list[dict]
    new_row = row.copy()
    new_row["description"] = clean_html_text(clean_summary(row.get("description")))
    gold_clean.append(new_row)

supa_df = [map_gold_row_to_job_listing(row) for row in gold_clean]
display(supa_df)
upload_gold_to_supabase(supa_df)


[{'url': 'https://www.governmentjobs.com/jobs/5108053-0/county-engineer',
  'apply_link': 'https://www.governmentjobs.com/jobs/5108053-0/county-engineer',
  'job_title': 'County Engineer',
  'company_name': 'New Hanover County',
  'job_location': 'New Hanover County, NC',
  'competitiveness_score': None,
  'country_code': 'US',
  'job_seniority_level': 'Entry level',
  'job_employment_type': 'full-time',
  'job_industries': ['Engineering', 'Government'],
  'job_base_pay_range': '111570 - 186699 YEAR',
  'job_summary': 'New Hanover County is seeking an experienced, licensed Professional Engineer to serve as County Engineer, providing leadership and direction of the operations and services within the Engineering department. The County Engineer serves as the department head and subject matter expert, overseeing the department\'s projects and activities, compliance with ordinances, and completion of subdivision reviews, capital projects, stormwater operations, and shore protection efforts.

Uploading 55 rows to job_listings via REST...
Batch 0-55 uploaded.


In [50]:
def fetch_job_listings_last_7_days():
    # 1) Compute cutoff timestamp (UTC, 7 days ago)
    cutoff = datetime.now(timezone.utc) - timedelta(days=7)
    cutoff_iso = cutoff.isoformat()

    # 2) Build REST URL with filter: job_posted_date >= cutoff
    # PostgREST syntax: ?job_posted_date=gte.<value>
    url = (
        f"{SUPABASE_URL}/rest/v1/{TABLE_NAME}"
        f"?select=*"
        f"&job_posted_date=gte.{cutoff_iso}"
    )

    headers = {
        "apikey": SUPABASE_KEY,
        "Authorization": f"Bearer {SUPABASE_KEY}",
    }

    resp = requests.get(url, headers=headers)
    try:
        data = resp.json()
    except Exception:
        print("❌ Could not parse JSON:", resp.status_code, resp.text)
        return None

    if isinstance(data, dict) and "message" in data:
        print("❌ Supabase error:", data)
        return None

    return pd.DataFrame(data)

# Fetch & display jobs from last 7 days
df_last_7 = fetch_job_listings_last_7_days()
import html
import re

import html
import re

def clean_html_text(text):
    if not text:
        return None
    
    # Fix broken entities like "&nb." → " "
    text = re.sub(r"&nb[^;]*;?", " ", text)
    
    # Decode proper HTML entities (&nbsp;, &amp;, etc.)
    decoded = html.unescape(text)

    # Strip extra whitespace
    return decoded.strip()

def clean_summary(text):
    if text is None:
        return None
    # 1. Convert HTML entities like &lt; &gt; &quot;
    decoded = html.unescape(text)
    # 2. Remove all HTML tags
    cleaned = re.sub(r"<[^>]+>", "", decoded)
    # 3. Trim whitespace
    return cleaned.strip()

df_clean = df_last_7.copy()
df_clean["job_summary"] = df_clean["job_summary"].apply(clean_summary).apply(clean_html_text)

df_clean


,id,url,job_title,company_name,job_location,country_code,job_seniority_level,job_employment_type,job_industries,job_summary,apply_link,job_base_pay_range,job_posted_date,competitiveness_score
0,10244,https://www.governmentjobs.com/jobs/5146646-0/...,Management Analyst,City of Modesto,"Tenth Street Place - 1010 10th Street Modesto, CA",None,None,None,None,Are you ready to make a meaningful impact in y...,https://www.governmentjobs.com/jobs/5146646-0/...,None,2025-11-21,None
1,10246,https://www.governmentjobs.com/jobs/5151057-0/...,Analyst I/II (Employment) - Human Resources De...,City of San Jose,San Jose,None,None,None,None,The Human Resources Department delivers innova...,https://www.governmentjobs.com/jobs/5151057-0/...,None,2025-11-21,None
2,10253,https://www.governmentjobs.com/jobs/5152573-0/...,Defense Strategy Data Analyst (Crime Analyst),Pinal County,971 Jason Lopez Circle Bldg G Florence AZ 8513...,None,None,None,None,Are you ready to use the power of data to driv...,https://www.governmentjobs.com/jobs/5152573-0/...,None,2025-11-23,None
3,10254,https://www.governmentjobs.com/jobs/5150270-0/...,Business Analyst (Information Technology Analy...,New Jersey Courts,"Trenton, NJ",None,None,None,None,If another position becomes available within f...,https://www.governmentjobs.com/jobs/5150270-0/...,None,2025-11-20,None
4,10058,https://www.governmentjobs.com/jobs/5148422-0/...,Software Application Specialist,"Washington County, Oregon","Hillsboro, OR",None,None,None,None,"The current vacancy is a non-benefited, variab...",https://www.governmentjobs.com/jobs/5148422-0/...,None,2025-11-22,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,10851,https://www.governmentjobs.com/jobs/5151057-0/...,Analyst I/II (Employment) - Human Resources De...,City of San Jose,,US,Entry level,full-time,"[""Data Science""]",The Human Resources Department delivers innova...,https://www.governmentjobs.com/jobs/5151057-0/...,92994.72 - 123330.48 YEAR,2025-11-21,None
188,10858,https://www.governmentjobs.com/jobs/5152573-0/...,Defense Strategy Data Analyst (Crime Analyst),Pinal County,,US,Entry level,fed,"[""Software"", ""Data Science""]",Are you ready to use the power of data to driv...,https://www.governmentjobs.com/jobs/5152573-0/...,None,2025-11-23,None
189,10859,https://www.governmentjobs.com/jobs/5150270-0/...,Business Analyst (Information Technology Analy...,New Jersey Courts,,US,Entry level,internship,"[""Engineering"", ""Software"", ""Data Science""]",If another position becomes available within f...,https://www.governmentjobs.com/jobs/5150270-0/...,69501.89 - 117971.38 YEAR,2025-11-20,None
190,10861,https://www.governmentjobs.com/jobs/5150948-0/...,TAA System Analyst (Business Analyst - Journey),State of Washington,,US,Entry level,internship,"[""Software"", ""Business Analysis""]",This recruitment is only open to Department of...,https://www.governmentjobs.com/jobs/5150948-0/...,None,2025-11-24,None
